# 🎮 Demo Rápido: Widget Interactivo del Mapa Logístico

Este notebook contiene únicamente el código necesario para probar el widget interactivo.

**Instrucciones:**
1. Ejecuta la primera celda (instalación e importación de librerías)
2. Ejecuta la segunda celda (widget interactivo)
3. ¡Experimenta con los controles!

---

In [ ]:
# ============================================
# INSTALACIÓN E IMPORTACIÓN DE LIBRERÍAS
# ============================================

# --- Import packages ---
try:
    import micropip
    await micropip.install(['ipywidgets', 'bqplot', 'plotly'])
except (ImportError, AttributeError, NameError):
    import sys
    import subprocess
    subprocess.check_call([sys.executable, '-m', 'pip', 'install', 'ipywidgets', 'bqplot', 'plotly'])

# --- Test imports ---
try:
    import ipywidgets as widgets
    print("✓ ipywidgets importado correctamente")
except ImportError:
    print("✗ Error al importar ipywidgets")

try:
    import bqplot as bq
    print("✓ bqplot importado correctamente")
except ImportError:
    print("✗ Error al importar bqplot")

try:
    import plotly.graph_objects as go
    print("✓ plotly importado correctamente")
except ImportError:
    print("✗ Error al importar plotly")

import numpy as np
import matplotlib
print("✓ numpy y matplotlib disponibles")

# --- JavaScript patch for bqplot dragging ---
from IPython.display import display, Javascript
display(Javascript("""
// Patch for bqplot Scatter.enable_move in JupyterLite
(function() {
    if (typeof window.bqplotScatterPatchApplied !== 'undefined') return;
    
    function waitForBqplot() {
        if (typeof window.require === 'undefined') {
            setTimeout(waitForBqplot, 100);
            return;
        }
        
        require(['bqplot'], function(bqplot) {
            if (!bqplot || !bqplot.Scatter || !bqplot.Scatter.prototype) {
                console.warn('bqplot.Scatter no encontrado para parchear');
                return;
            }
            
            const originalInitialize = bqplot.Scatter.prototype.initialize;
            bqplot.Scatter.prototype.initialize = function(options) {
                const result = originalInitialize.apply(this, arguments);
                if (this.model.get('enable_move')) {
                    this._enableDrag();
                }
                return result;
            };
            
            if (!bqplot.Scatter.prototype._enableDrag) {
                bqplot.Scatter.prototype._enableDrag = function() {
                    if (!this.d3el || !this.d3el.selectAll) return;
                    
                    const that = this;
                    this.d3el.selectAll('.object_grp')
                        .style('cursor', 'move')
                        .call(d3.drag()
                            .on('drag', function(event, d) {
                                const scales = that.model.get('scales');
                                const xScale = scales.x;
                                const yScale = scales.y;
                                
                                if (xScale && yScale) {
                                    const newX = xScale.scale.invert(event.x);
                                    const newY = yScale.scale.invert(event.y);
                                    
                                    that.model.set('x', [newX]);
                                    that.model.set('y', [newY]);
                                    that.model.save_changes();
                                }
                            })
                        );
                };
            }
            
            window.bqplotScatterPatchApplied = true;
            console.log('✓ Parche de arrastre bqplot aplicado correctamente');
        }, function(err) {
            console.warn('No se pudo cargar bqplot para parchear:', err);
        });
    }
    
    if (document.readyState === 'loading') {
        document.addEventListener('DOMContentLoaded', waitForBqplot);
    } else {
        waitForBqplot();
    }
})();
"""))

print("\n🎉 ¡Todo listo! Ejecuta la siguiente celda para el widget interactivo.")

# --- Function definitions ---
def mapa_logistico_extendido(r, x_n_minus_1, x_n, n_pasos):
    """
    Implementa el mapa logístico extendido: x_{n+1} = r * x_n * (1 - x_{n-1})
    
    Args:
        r: Parámetro del mapa (típicamente 0 < r ≤ 4)
        x_n_minus_1: Condición inicial x_0
        x_n: Condición inicial x_1
        n_pasos: Número de iteraciones
    
    Returns:
        Lista con la secuencia completa [x_0, x_1, x_2, ..., x_n]
    """
    if n_pasos < 2:
        raise ValueError("n_pasos debe ser al menos 2 para el mapa extendido")
    
    valores = [x_n_minus_1, x_n]
    
    for i in range(n_pasos - 2):
        x_nuevo = r * valores[-1] * (1 - valores[-2])
        valores.append(x_nuevo)
    
    return valores

def generar_secuencia(r, x0, n):
    """
    Genera secuencia del mapa logístico normal: x_{n+1} = r * x_n * (1 - x_n)
    
    Args:
        r: Parámetro del mapa
        x0: Condición inicial
        n: Número de iteraciones
    
    Returns:
        Lista con la secuencia [x_0, x_1, ..., x_n]
    """
    valores = [x0]
    for i in range(n - 1):
        x_nuevo = r * valores[-1] * (1 - valores[-1])
        valores.append(x_nuevo)
    return valores

def construir_pares_fase(valores):
    """
    Construye pares (x_{n-1}, x_n) para graficar en espacio de fases
    
    Args:
        valores: Lista de valores de la secuencia
    
    Returns:
        Array numpy de forma (n, 2) con pares [x_{n-1}, x_n]
    """
    pares = []
    for i in range(len(valores) - 1):
        pares.append([valores[i], valores[i+1]])
    return np.array(pares)

In [ ]:
# ============================================
# WIDGET INTERACTIVO
# ============================================

import plotly.graph_objects as go
import ipywidgets as widgets
from IPython.display import display, HTML
import numpy as np
import matplotlib

# Try importing bqplot (optional for draggable mode)
try:
    import bqplot as bq
    from matplotlib import cm
    BQPLOT_AVAILABLE = True
except ImportError:
    BQPLOT_AVAILABLE = False
    print("bqplot no disponible - modo arrastrable deshabilitado")

# --- Mode toggle ---
mode_toggle = widgets.ToggleButtons(
    options=['Sliders', 'Arrastrable'],
    value='Sliders',
    description='Modo:',
    disabled=not BQPLOT_AVAILABLE,
    button_style='info'
)

# --- Sliders ---
r_slider = widgets.FloatSlider(value=2.0, min=0.0, max=4.0, step=0.01, description='r')
x0_slider = widgets.FloatSlider(value=0.5, min=0.0, max=1.0, step=0.01, description='x0')
x1_slider = widgets.FloatSlider(value=0.5, min=0.0, max=1.0, step=0.01, description='x1')
N_slider = widgets.IntSlider(value=20, min=3, max=200, step=1, description='N')

# --- Map type buttons ---
map_log_button = widgets.ToggleButton(value=True, description="Mapa logístico", layout=widgets.Layout(width='180px'))
map_ext_button = widgets.ToggleButton(value=False, description="Mapa extendido", layout=widgets.Layout(width='180px'))
cobweb_button = widgets.ToggleButton(value=False, description="Cobweb", layout=widgets.Layout(width='120px'))
compare_button = widgets.Button(description="Comparar evoluciones", layout=widgets.Layout(width='220px'))
reset_button = widgets.Button(description="Reset", button_style='warning', layout=widgets.Layout(width='100px'))

# --- Output widgets ---
plot_output = widgets.Output()
time_output = widgets.Output()
compare_output = widgets.Output()
info_output = widgets.HTML()

# --- Internal state ---
x0_val = 0.5
x1_val = 0.5
updating = False
bqplot_displayed = False

# --- bqplot objects (only if available) ---
if BQPLOT_AVAILABLE:
    x_sc = bq.LinearScale(min=0, max=1)
    y_sc = bq.LinearScale(min=0, max=1)
    c_sc = bq.ColorScale(scheme='viridis')  # lowercase for consistency
    
    x_parab = np.linspace(0, 1, 300)
    y_parab = r_slider.value * x_parab * (1 - x_parab)
    
    parabola = bq.Lines(x=x_parab, y=y_parab, scales={'x': x_sc, 'y': y_sc},
                        colors=['gray'], opacities=[0.3], line_style='dotted')
    diagonal = bq.Lines(x=[0, 1], y=[0, 1], scales={'x': x_sc, 'y': y_sc},
                       colors=['gray'], line_style='dashed', labels=['y=x'])
    scatter = bq.Scatter(x=[], y=[], color=[], scales={'x': x_sc, 'y': y_sc, 'color': c_sc},
                        default_size=64, stroke='white', stroke_width=0.5)
    initial_point = bq.Scatter(x=[x0_val], y=[x1_val], scales={'x': x_sc, 'y': y_sc},
                              colors=['blue'], default_size=150, enable_move=True)
    final_point = bq.Scatter(x=[], y=[], scales={'x': x_sc, 'y': y_sc},
                            colors=['red'], default_size=150)
    
    ax_x = bq.Axis(scale=x_sc, label='x_{n-1}', grid_lines='solid')
    ax_y = bq.Axis(scale=y_sc, label='x_n', orientation='vertical', grid_lines='solid')
    ax_c = bq.ColorAxis(scale=c_sc, label='Iteraciones', orientation='vertical', side='right')
    
    fig_bqplot = bq.Figure(marks=[parabola, diagonal, scatter, final_point, initial_point], axes=[ax_x, ax_y, ax_c],
                          title='Espacio de fases (arrastra punto azul)',
                          fig_margin={'top': 60, 'bottom': 60, 'left': 60, 'right': 100})
    fig_bqplot.layout.width = '700px'
    fig_bqplot.layout.height = '650px'
    
    cobweb_marks = []

# --- Toggle map type ---
def toggle_maps(change=None):
    if change['owner'] == map_log_button and map_log_button.value:
        map_ext_button.value = False
    elif change['owner'] == map_ext_button and map_ext_button.value:
        map_log_button.value = False

map_log_button.observe(toggle_maps, names='value')
map_ext_button.observe(toggle_maps, names='value')

# --- Generate sequence ---
def generar_secuencia_interna(r, x0, x1, n, use_extended):
    if use_extended:
        if n < 2:
            raise ValueError("n_pasos debe ser al menos 2")
        valores = [x0, x1]
        for _ in range(n - 2):
            nuevo = r * valores[-1] * (1 - valores[-2])
            valores.append(nuevo)
        return valores
    else:
        valores = [x0]
        for _ in range(n - 1):
            valores.append(r * valores[-1] * (1 - valores[-1]))
        return valores

# --- Core update function ---
def actualizar_interfaz(change=None):
    global updating, x0_val, x1_val, cobweb_marks, bqplot_displayed
    
    if updating:
        return
    updating = True
    
    r_val = r_slider.value
    N_val = max(2, N_slider.value)
    use_extended = map_ext_button.value
    
    # Get x0, x1 from appropriate source and sync with other controls
    if mode_toggle.value == 'Arrastrable' and BQPLOT_AVAILABLE:
        # Read from draggable point
        x0_val = float(initial_point.x[0])
        x1_val = float(initial_point.y[0])
        # Sync sliders (without triggering their observers due to updating flag)
        x0_slider.value = x0_val
        if use_extended:
            x1_slider.value = x1_val
    else:
        # Read from sliders
        x0_val = x0_slider.value
        x1_val = x1_slider.value
        # Sync draggable point if available
        if BQPLOT_AVAILABLE:
            initial_point.x = [x0_val]
            initial_point.y = [x1_val]
    
    # If logistic map, constrain x1 and update both slider and draggable point
    if not use_extended:
        x1_val = r_val * x0_val * (1 - x0_val)
        x1_slider.value = x1_val
        if BQPLOT_AVAILABLE:
            initial_point.y = [x1_val]
    
    # Generate sequence
    try:
        valores = generar_secuencia_interna(r_val, x0_val, x1_val, N_val, use_extended)
    except (ValueError, OverflowError) as e:
        info_output.value = f"<b>Error:</b> {str(e)}"
        updating = False
        return
    
    pares = construir_pares_fase(valores)
    colores = np.linspace(0, N_val, len(valores))
    
    # --- Update info ---
    info_html = f"<b>Total de valores:</b> {len(valores)}<br>"
    info_html += "<b>Primeros valores (hasta 10):</b><br>"
    for idx, val in enumerate(valores[:10]):
        info_html += f"x_{idx} = {val:.5f}<br>"
    info_output.value = info_html
    
    # --- Update appropriate plot ---
    if mode_toggle.value == 'Arrastrable' and BQPLOT_AVAILABLE:
        # Use hold_sync for smooth updates
        with fig_bqplot.hold_sync():
            # Update parabola
            y_parab_new = r_val * x_parab * (1 - x_parab)
            parabola.y = y_parab_new
            
            # Initial point is already synced above
            
            # Update scatter
            scatter.x = pares[:, 0]
            scatter.y = pares[:, 1]
            scatter.color = colores
            c_sc.min = 0
            c_sc.max = N_val
            
            # Update final point
            final_point.x = [pares[-1, 0]]
            final_point.y = [pares[-1, 1]]
            
            # Handle cobweb - update in place or hide/show
            if cobweb_button.value and len(pares) > 0:
                viridis = matplotlib.colormaps['viridis']
                # First horizontal + all remaining segments (skip only first vertical)
                needed_lines = 1 + 2 * (len(pares) - 2)
                
                # Reuse existing cobweb marks if possible
                if len(cobweb_marks) == needed_lines:
                    # Update existing marks in place
                    mark_idx = 0
                    for i in range(len(pares) - 1):
                        norm_color = colores[i+1] / N_val if N_val > 0 else 0
                        rgba = viridis(norm_color)
                        hex_color = '#{:02x}{:02x}{:02x}'.format(int(rgba[0]*255), int(rgba[1]*255), int(rgba[2]*255))
                        
                        # Skip first vertical (i=0, vertical)
                        if i > 0:
                            # Update vertical line
                            cobweb_marks[mark_idx].x = [pares[i,0], pares[i,0]]
                            cobweb_marks[mark_idx].y = [pares[i,0], pares[i,1]]
                            cobweb_marks[mark_idx].colors = [hex_color]
                            mark_idx += 1
                        
                        # Update horizontal line (including first one)
                        cobweb_marks[mark_idx].x = [pares[i,0], pares[i,1]]
                        cobweb_marks[mark_idx].y = [pares[i,1], pares[i,1]]
                        cobweb_marks[mark_idx].colors = [hex_color]
                        mark_idx += 1
                else:
                    # Need to recreate marks (size changed)
                    if cobweb_marks:
                        current_marks = list(fig_bqplot.marks)
                        for mark in cobweb_marks:
                            if mark in current_marks:
                                current_marks.remove(mark)
                        fig_bqplot.marks = current_marks
                        cobweb_marks = []
                    
                    new_cobweb = []
                    # Create cobweb segments (skip only first vertical)
                    for i in range(len(pares) - 1):
                        norm_color = colores[i+1] / N_val if N_val > 0 else 0
                        rgba = viridis(norm_color)
                        hex_color = '#{:02x}{:02x}{:02x}'.format(int(rgba[0]*255), int(rgba[1]*255), int(rgba[2]*255))
                        
                        # Skip first vertical (i=0, vertical line)
                        if i > 0:
                            vert = bq.Lines(x=[pares[i,0], pares[i,0]], y=[pares[i,0], pares[i,1]],
                                           scales={'x': x_sc, 'y': y_sc}, colors=[hex_color], opacities=[0.6])
                            new_cobweb.append(vert)
                        
                        # Always add horizontal line (including first one)
                        horiz = bq.Lines(x=[pares[i,0], pares[i,1]], y=[pares[i,1], pares[i,1]],
                                        scales={'x': x_sc, 'y': y_sc}, colors=[hex_color], opacities=[0.6])
                        new_cobweb.append(horiz)
                    
                    cobweb_marks = new_cobweb
                    fig_bqplot.marks = [parabola, diagonal] + cobweb_marks + [scatter, final_point, initial_point]
            else:
                # Cobweb disabled - remove marks if present
                if cobweb_marks:
                    current_marks = list(fig_bqplot.marks)
                    for mark in cobweb_marks:
                        if mark in current_marks:
                            current_marks.remove(mark)
                    fig_bqplot.marks = current_marks
                    cobweb_marks = []
        
        # Display bqplot only once
        if not bqplot_displayed:
            with plot_output:
                plot_output.clear_output(wait=True)
                display(fig_bqplot)
            bqplot_displayed = True
            
    else:
        # Reset bqplot display flag when switching to plotly
        bqplot_displayed = False
        
        # Update plotly
        with plot_output:
            plot_output.clear_output(wait=True)
            
            fig = go.Figure()
            
            # Diagonal
            fig.add_scatter(x=[0, 1], y=[0, 1], mode='lines',
                          line=dict(color='gray', dash='dash', width=2), name='Diagonal y=x')
            
            # Parabola (if logistic)
            if not use_extended:
                x_par = np.linspace(0, 1, 200)
                y_par = r_val * x_par * (1 - x_par)
                fig.add_scatter(x=x_par, y=y_par, mode='lines',
                              line=dict(color='gray', dash='dot', width=1), name='y=rx(1-x)', opacity=0.3)
            
            # Cobweb (skip only first vertical line) - use Viridis colors
            if cobweb_button.value and len(pares) > 0:
                import matplotlib.colors as mcolors
                viridis = matplotlib.colormaps['viridis']
                
                for i in range(len(pares)-1):
                    # Get color from Viridis colormap
                    norm_color = colores[i+1] / N_val if N_val > 0 else 0
                    rgba = viridis(norm_color)
                    hex_color = mcolors.rgb2hex(rgba[:3])
                    
                    # Skip first vertical (i=0, vertical line)
                    if i > 0:
                        fig.add_scatter(x=[pares[i,0], pares[i,0]], y=[pares[i,0], pares[i,1]],
                                      mode='lines', line=dict(color=hex_color, width=1), showlegend=False, opacity=0.6)
                    # Always add horizontal (including first one)
                    fig.add_scatter(x=[pares[i,0], pares[i,1]], y=[pares[i,1], pares[i,1]],
                                  mode='lines', line=dict(color=hex_color, width=1), showlegend=False, opacity=0.6)
            
            # Trajectory
            fig.add_scatter(x=pares[:, 0], y=pares[:, 1], mode='markers',
                          marker=dict(size=8, color=colores, colorscale='Viridis', showscale=True,
                                     colorbar=dict(title='Iteraciones',
                                                  tickvals=np.linspace(0, N_val, min(6, N_val+1)),
                                                  ticktext=[f"{int(t)}" for t in np.linspace(0, N_val, min(6, N_val+1))])),
                          name='Trayectoria', showlegend=False)
            
            # Final point
            fig.add_scatter(x=[pares[-1, 0]], y=[pares[-1, 1]], mode='markers+text',
                            marker=dict(size=12, color='red'),
                            text=['Último punto'],
                            textposition='bottom left',
                            showlegend=False
)
                        
            # Initial point
            fig.add_scatter(x=[pares[0, 0]], y=[pares[0, 1]], mode='markers+text',
                            marker=dict(size=12,color='blue'),
                            text=['Inicio'],
                            textposition='top right',
                            showlegend=False
)
            
            fig.update_layout(title='Trayectoria en el espacio de fases',
                            xaxis=dict(title='x<sub>n-1</sub>', range=[0,1], gridcolor='lightgray', showgrid=True),
                            yaxis=dict(title='x<sub>n</sub>', range=[0,1], gridcolor='lightgray', showgrid=True),
                            width=700, height=650, hovermode='closest',legend=dict(
                                                                                x=0.02, y=0.98,          # position (top-left)
                                                                                xanchor='left', yanchor='top',
                                                                                bgcolor='rgba(255,255,255,0.6)',
                                                                                bordercolor='gray',
                                                                                borderwidth=1
                                                                                )
                             )
            
            display(HTML(fig.to_html(include_plotlyjs='cdn')))
    
    # --- Time evolution ---
    with time_output:
        time_output.clear_output(wait=True)
        fig_time = go.Figure()
        fig_time.add_scatter(x=np.arange(len(valores)), y=valores, mode='markers',
                           marker=dict(size=8, color=colores, colorscale='Viridis', showscale=True))
        fig_time.update_layout(title='Evolución temporal', xaxis=dict(title='n'),
                             yaxis=dict(title='x_n', range=[0,1]), width=700, height=400)
        display(HTML(fig_time.to_html(include_plotlyjs='cdn')))
    
    updating = False

# --- Drag handler for bqplot ---
if BQPLOT_AVAILABLE:
    def on_drag(change):
        if change['name'] not in ['x', 'y'] or updating:
            return
        actualizar_interfaz()
    
    initial_point.observe(on_drag, names=['x', 'y'])

# --- Mode toggle handler ---
def on_mode_change(change):
    global bqplot_displayed
    slider_box.layout.visibility = 'visible' if change['new'] == 'Sliders' else 'hidden'
    bqplot_displayed = False  # Reset flag when switching modes
    actualizar_interfaz()

mode_toggle.observe(on_mode_change, names='value')

# --- Reset ---
def reset_interface(b):
    global updating, x0_val, x1_val, bqplot_displayed
    updating = True
    r_slider.value = 2.0
    x0_slider.value = 0.5
    x1_slider.value = 0.5
    N_slider.value = 20
    map_log_button.value = True
    map_ext_button.value = False
    cobweb_button.value = False
    x0_val = 0.5
    x1_val = 0.5
    if BQPLOT_AVAILABLE:
        initial_point.x = [0.5]
        initial_point.y = [0.5]
    bqplot_displayed = False
    updating = False
    actualizar_interfaz()

# --- Compare ---
def comparar_evoluciones(b):
    compare_output.clear_output(wait=True)
    r_val = r_slider.value
    N_val = N_slider.value
    
    x_log = generar_secuencia_interna(r_val, x0_val, x1_val, N_val, False)
    x_ext = generar_secuencia_interna(r_val, x0_val, x1_val, N_val, True)
    colores = np.linspace(0, N_val, len(x_ext))
    
    with compare_output:
        fig_comp = go.Figure()
        fig_comp.add_scatter(x=np.arange(len(x_log)), y=x_log, mode='markers',
                           marker=dict(size=8, color=colores, colorscale='Viridis'), name='Normal')
        fig_comp.add_scatter(x=np.arange(len(x_ext)), y=x_ext, mode='markers',
                           marker=dict(size=8, color=colores, colorscale='Plasma'), name='Extendido')
        fig_comp.update_layout(title='Comparación: Normal vs Extendido', xaxis=dict(title='n'),
                             yaxis=dict(title='x_n', range=[0,1]), width=800, height=400)
        display(HTML(fig_comp.to_html(include_plotlyjs='cdn')))

# --- Attach observers ---
for ctrl in (r_slider, N_slider, map_log_button, map_ext_button, cobweb_button):
    ctrl.observe(actualizar_interfaz, names='value')
for ctrl in (x0_slider, x1_slider):
    ctrl.observe(lambda c: actualizar_interfaz() if mode_toggle.value == 'Sliders' else None, names='value')

compare_button.on_click(comparar_evoluciones)
reset_button.on_click(reset_interface)

# --- Initial display ---
actualizar_interfaz()

# --- Layout ---
slider_box = widgets.VBox([x0_slider, x1_slider])
map_buttons = widgets.HBox([map_log_button, map_ext_button, cobweb_button, compare_button, reset_button])
controls = widgets.VBox([mode_toggle, r_slider, N_slider, slider_box, map_buttons])
display(widgets.VBox([controls, plot_output, time_output, compare_output, info_output]))